In [ ]:
%pip install h5py

In [ ]:
import librosa
import numpy as np
import os
import glob
from pathlib import Path

In [ ]:
VOCAL_DIR = '../media_files/separated_audio/train/vocals'
INSTRUMENTAL_DIR = '../media_files/separated_audio/train/instrumentals'
OUTPUT_DIR= '../media_files/preprocessed_audio/train'

In [ ]:
# --- Cập nhật tham số mới ---
SR = 44100
N_FFT = 2048
HOP_LENGTH = 512

# Tăng kích thước cửa sổ lên 128 (frames)
WINDOW_SIZE = 128 

# Bước nhảy nên để 50% cửa sổ để tạo độ chồng lấp (overlap), giúp data mượt hơn
WINDOW_STEP = 64

In [ ]:
def perform_stft(y_vocal, y_inst):
    """
    Using librosa to perform stft on the audio file
    """
    
    # --- ĐOẠN SỬA LỖI ---
    # 1. Tìm độ dài ngắn nhất giữa 2 file
    min_length = min(len(y_vocal), len(y_inst))
    
    # 2. Cắt cả 2 file về độ dài ngắn nhất để khớp kích thước
    y_vocal = y_vocal[:min_length]
    y_inst = y_inst[:min_length]
    # --------------------

    # 3. Tạo mixture (bản mix hỗn hợp)
    y_mix = y_vocal + y_inst
    
    # 4. Biến đổi STFT (Lấy phần biên độ - Magnitude)
    # Kết quả sẽ có 1025 hàng (bins)
    mag_vocal = np.abs(librosa.stft(y_vocal, n_fft=N_FFT, hop_length=HOP_LENGTH))
    mag_inst = np.abs(librosa.stft(y_inst, n_fft=N_FFT, hop_length=HOP_LENGTH))
    mag_mix = np.abs(librosa.stft(y_mix, n_fft=N_FFT, hop_length=HOP_LENGTH))

    return mag_vocal, mag_inst, mag_mix

In [ ]:
def generate_ibm(mag_vocal, mag_inst):
    ibm = (mag_vocal > mag_inst).astype(np.float32)
    return ibm

In [ ]:
def processing_audio(mag_mix, ibm):
    X_train = [] # Input: Spectrogram (Bản mix)
    Y_train = [] # Target: Mask (Vocal)

    # Chuẩn hóa biên độ về 0-1
    mag_mix_norm = mag_mix / np.max(mag_mix)
    
    # Cắt bỏ dòng tần số thứ 1025, chỉ lấy 1024 dòng đầu để dễ chia cho U-Net
    # Shape mới: (1024, frames)
    mag_mix_compressed = np.sqrt(mag_mix)
    mag_mix_norm = mag_mix_compressed / np.max(mag_mix_compressed)
    ibm = ibm[:1024, :]
    
    n_frames = mag_mix.shape[1]
    
    for i in range(0, n_frames - WINDOW_SIZE + 1, WINDOW_STEP):
        # Cắt cửa sổ (1024 x 20)
        x_window = mag_mix_norm[:, i : i + WINDOW_SIZE]
        y_window = ibm[:, i : i + WINDOW_SIZE]
        
        # Thêm 1 chiều (Channel) ở cuối để thành (1024, 20, 1) - Chuẩn input cho CNN/U-Net
        # KHÔNG DÙNG FLATTEN NỮA
        X_train.append(x_window[..., np.newaxis])
        Y_train.append(y_window[..., np.newaxis])
        
    return np.array(X_train), np.array(Y_train)

In [ ]:
vocal_directory = Path(VOCAL_DIR)
instrumental_directory = Path(INSTRUMENTAL_DIR)
pattern = "*.wav"   

In [ ]:
vocal_file_count = len(list(vocal_directory.glob(pattern)))
print(vocal_file_count)

In [ ]:
instrumental_file_count = len(list(instrumental_directory.glob(pattern)))
print(instrumental_file_count)

In [ ]:
# Since not all files have both vocal and instrumental, we keep track of valid pairs
valid_file_name = []

In [ ]:
def match_vocal_and_instrumental(vocal_dir, instrumental_dir):
    matched_pairs = []
    
    # Lấy danh sách tất cả file trong thư mục instrumental
    instrumental_files = [f for f in os.listdir(instrumental_dir) if f.endswith('.wav')]
    
    for filename in instrumental_files:
        # Đường dẫn đầy đủ của file instrumental
        inst_path = os.path.join(instrumental_dir, filename)
        
        # Tìm file có tên tương ứng trong thư mục vocal
        vocal_path = os.path.join(vocal_dir, filename)
        
        # Kiểm tra sự tồn tại của file vocal tương ứng
        if os.path.exists(vocal_path):
            matched_pairs.append({
                'name': filename,
                'vocal_path': vocal_path,
                'inst_path': inst_path
            })
        else:
            print(f"Cảnh báo: Không tìm thấy Vocal cho {filename}")
            
    return matched_pairs

# Sử dụng hàm
matched_data = match_vocal_and_instrumental(VOCAL_DIR, INSTRUMENTAL_DIR)
print(f"Đã khớp thành công {len(matched_data)} cặp file.")

In [ ]:
print(matched_data[0])

In [ ]:
final_data = []

In [ ]:
for data in matched_data:
    print(f"Vocal file processing: {data['vocal_path']} and Instrumental file processing: {data['inst_path']}")
    y_vocal, _ = librosa.load(data['vocal_path'], sr=SR, mono=True)
    y_inst, _ = librosa.load(data['inst_path'], sr=SR, mono=True)

    mag_vocal, mag_inst, mag_mix = perform_stft(y_vocal, y_inst)
    ibm = generate_ibm(mag_vocal, mag_inst)
    X_train, Y_train = processing_audio(mag_mix, ibm)

    final_data.append({
        'X_train': X_train,
        'Y_train': Y_train
    })

In [ ]:
print(final_data[0])

In [ ]:
y_train = final_data[0]['Y_train']

total_ones = np.sum(y_train == 1)
percentage = (total_ones / y_train.size) * 100

print(f"Tổng số điểm là Vocal (số 1): {total_ones}")
print(f"Tỷ lệ Vocal trong dữ liệu: {percentage:.2f}%")

In [ ]:
print(len(final_data))

In [ ]:
import h5py
import numpy as np

# Đường dẫn file output
output_path = 'data_2d.h5'

# Mở file h5 để ghi
with h5py.File(output_path, 'w') as f:
    # 1. Tạo dataset rỗng ban đầu
    # maxshape=(None, ...) nghĩa là chiều đầu tiên (số lượng mẫu) có thể mở rộng vô tận
    # Chunks=True giúp tối ưu việc ghi/đọc dữ liệu lớn
    
    # Lấy shape mẫu từ phần tử đầu tiên để định nghĩa kích thước (trừ chiều số lượng mẫu)
    # Ví dụ: sample_shape sẽ là (1024, 128, 1)
    sample_shape_X = final_data[0]['X_train'].shape[1:] 
    sample_shape_Y = final_data[0]['Y_train'].shape[1:]

    dset_x = f.create_dataset('X_train', shape=(0, *sample_shape_X), 
                              maxshape=(None, *sample_shape_X), 
                              dtype='float32', compression="gzip")
    
    dset_y = f.create_dataset('Y_train', shape=(0, *sample_shape_Y), 
                              maxshape=(None, *sample_shape_Y), 
                              dtype='float32', compression="gzip")

    # 2. Duyệt qua từng bài hát và ghi nối đuôi vào file (Append)
    total_samples = 0
    
    for i, data in enumerate(final_data):
        x_chunk = data['X_train']
        y_chunk = data['Y_train']
        
        # Nếu chunk rỗng thì bỏ qua
        if len(x_chunk) == 0:
            continue
            
        num_new_samples = x_chunk.shape[0]
        
        # Mở rộng kích thước dataset trong file
        dset_x.resize(total_samples + num_new_samples, axis=0)
        dset_y.resize(total_samples + num_new_samples, axis=0)
        
        # Ghi dữ liệu mới vào phần vừa mở rộng
        dset_x[total_samples : total_samples + num_new_samples] = x_chunk
        dset_y[total_samples : total_samples + num_new_samples] = y_chunk
        
        total_samples += num_new_samples
        
        # Xóa biến tạm để giải phóng RAM ngay lập tức
        del x_chunk, y_chunk
        
        # In tiến độ để biết code đang chạy
        if (i + 1) % 10 == 0:
            print(f"Đã lưu {i + 1}/{len(final_data)} file. Tổng mẫu hiện tại: {total_samples}")

print(f"Hoàn tất! Tổng cộng {total_samples} mẫu đã được lưu vào {output_path}")

In [ ]:
import gc
gc.collect()